In [1]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2024-08-24 10:07:02.898725: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-08-24 10:07:02.898740: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-08-24 10:07:02.898745: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-08-24 10:07:02.898806: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-24 10:07:02.899111: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


True

In [2]:
import torch
import transformers

print(transformers.__version__)
device_name = None
if torch.backends.mps.is_available():
    device_name = "mps" #cuda
else:
    device_name = "cpu"

print("device_name : ",device_name)

4.42.4
device_name :  mps


/Users/tharhtet/.local/share/virtualenvs/fastapi-GYQVYN5v/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import pipeline,Pipeline

In [5]:
def load_text_model(device_name):
    pipe = pipeline("text-generation",
                    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
                    torch_dtype=torch.bfloat16,
                    device= device_name)

    return pipe


In [6]:
prompt = "what is tensorflow"
pipe = load_text_model(device_name)

In [7]:
def generate_text(pipe: Pipeline, prompt: str, temperature: float = 0.7) -> str:

    system_prompt = """
    Your name is ML bot and you are a helpful
    chatbot responsible for teaching  machine learning system to your users.
    Always respond in markdown.
    """


    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ] 
    prompt = pipe.tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    predictions = pipe(
        prompt,
        temperature=temperature,
        max_new_tokens=256,
        do_sample=True,
        top_k=50,
        top_p=0.95,
    ) 
    output = predictions[0]["generated_text"].split("</s>\n<|assistant|>\n")[-1]
    return output

In [8]:
prompt = " What is tensorflow "
generate_text = generate_text(pipe=pipe,prompt=prompt)

In [9]:
generate_text

' TensorFlow is a popular open-source machine learning library developed by Google that provides a flexible and efficient framework for developing and training deep learning models. It offers a wide range of tools and libraries for data processing, model training, and inference, making it an attractive choice for developers who are new to deep learning. TensorFlow is designed to provide a simple and intuitive user interface that makes it easy for developers to build and train deep learning models. With TensorFlow, you can create custom models, train them using a variety of algorithms, and deploy them to various platforms, such as cloud services like Google Cloud Platform (GCP) and AWS. TensorFlow is available as a Python library, as well as a command-line interface (CLI) for users who prefer to work with command-line tools.'